# Generate Supply (From date 2023-03-20 to 2023-12-27 to match demand data)

- Generated supply data is saved in data/network_data.yml

In [1]:
from src.data_generator.service_generator import ServiceGenerator

file_name = "../data/network_data.yml"
path_config = "../src/data_generator/config.yml"
n_services = 100
seed = 1

ServiceGenerator().generate(file_name = file_name,
                            path_config = path_config,
                            n_services = n_services,
                            seed = seed)

# Load generated data (Just to check everything is ok)

In [2]:
from src.robin.supply.entities import Supply

path_config_supply = "../data/network_data.yml"

network_supply = Supply.from_yaml(path_config_supply)

# Run kernel simulation using generated supply data

In [3]:
from src.robin.kernel.entities import Kernel

path_config_supply = "../data/network_data.yml"
path_config_demand = "../data/demand_data.yml"

kernel = Kernel(path_config_supply=path_config_supply,
                path_config_demand=path_config_demand)

services = kernel.simulate(output_path="../data/output.csv")

# Print services with tickets sold
for s in services:
    if any(seat_tickets != 0 for pair_tickets in s.seats_log.values() for seat_tickets in pair_tickets.values()):
        print(s)

Service id: 60000_71500_10111100_6:6_10 
	Date of service: 2023-03-23 
	Stops: ['60000', '70600', '04040', '78400', '71500'] 
	Line times (relative): [(0.0, 0.0), (31.2, 34.2), (49.8, 52.8), (68.4, 71.4), (87.0, 90.0)] 
	Line times (absolute): [('06:06', '06:06'), ('06:37', '06:40'), ('06:55', '06:58'), ('07:14', '07:17'), ('07:33', '07:36')] 
	Train Service Provider: [4, IRYO, ['41', '42']] 
	Time Slot: [6:6_10, 6:06:00, 6:16:00, 6:11:00, 0:10:00] 
	Rolling Stock: [41,OUIGO RollingStock 1,{1: 300, 2: 50}] 
	Prices: 
		('60000', '70600'): {1: 18.08, 2: 21.69, 3: 28.93}
		('60000', '04040'): {1: 21.22, 2: 25.46, 3: 33.95}
		('60000', '78400'): {1: 26.42, 2: 31.7, 3: 42.27}
		('60000', '71500'): {1: 28.07, 2: 33.68, 3: 44.9}
		('70600', '04040'): {1: 12.14, 2: 14.57, 3: 19.42}
		('70600', '78400'): {1: 17.62, 2: 21.15, 3: 28.2}
		('70600', '71500'): {1: 19.95, 2: 23.93, 3: 31.91}
		('04040', '78400'): {1: 14.78, 2: 17.73, 3: 23.64}
		('04040', '71500'): {1: 17.58, 2: 21.09, 3: 28.12}
		(

In [1]:
from src.data_generator.service_generator import ServiceGenerator
import pandas as pd

# Lista de servicios
file_name = "../data/network_data.yml"
path_config = "../src/data_generator/config.yml"
n_services = 100
seed = 1

services_list = ServiceGenerator().generate(file_name = file_name,
                            path_config = path_config,
                            n_services = n_services,
                            seed = seed)

# Creamos un DataFrame con los atributos relevantes
services_df = pd.DataFrame({
    'duration': [service.schedule[-1][1] - service.schedule[0][0] for service in services_list], # Duración del servicio
    'num_stops': [len(service.schedule) for service in services_list], # Número de paradas
    'price': [service.prices[('origin', 'destination')]['asiento1'] for service in services_list], # Precio del billete para un tipo de asiento
})

# Normalizamos los valores en el DataFrame
services_df = (services_df - services_df.min()) / (services_df.max() - services_df.min())

KeyError: ('origen', 'destino')

# Autoencoder

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# Define el número de características de entrada
input_dim = 3

# Define el número de neuronas en la capa oculta
encoding_dim = 2

# Define la entrada
input_data = Input(shape=(input_dim,))

# Define la capa codificadora
encoded = Dense(encoding_dim, activation='relu')(input_data)

# Define la capa decodificadora
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Define el modelo del autoencoder
autoencoder = Model(input_data, decoded)

# Define el modelo del codificador
encoder = Model(input_data, encoded)

# Define el modelo del decodificador
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
